In [18]:
import plotly.graph_objs as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import os
import numpy as np

In [3]:
os.chdir(os.pardir)
os.getcwd()

'/Users/Adem/Python_Projects/long-live-the-battery'

In [136]:
from trainer.data_pipeline import load_scaling_factors
import trainer.constants as cst

In [6]:
scaling_factors = load_scaling_factors()
scaling_factors

{'IR': 0.022039292,
 'QD': 1.0979686,
 'Remaining_cycles': 2159.0,
 'Discharge_time': 14.758193333333232,
 'Qdlin': 1.0828815,
 'Tdlin': 41.78867160669522}

In [421]:
mean_cycle_life = 1500

pred_current_cycle = 300
pred_remaining_cycles = 1200

results = np.array([
    pred_current_cycle / scaling_factors["Remaining_cycles"],
    pred_remaining_cycles / scaling_factors["Remaining_cycles"]
]).astype(np.float32)

def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))

def sigmoid_mod_normalized(x):  
    sigmoid_mod = np.exp(-np.logaddexp(0, (-x + 5)*1.2))
    return (sigmoid_mod - sigmoid_mod.min()) / (sigmoid_mod.max() - sigmoid_mod.min())

def visualize_results(results, window_size, scaling_factors, mean_cycle_life, inline=False):
    relative_results = results / results.sum()  # Relative to this cell
    results_cycle_life = np.sum(np.round(
        results * scaling_factors[cst.REMAINING_CYCLES_NAME]).astype(np.int))
    relative_window_size = float(window_size / results_cycle_life)
    print("relative results", relative_results)
    print("window", relative_window_size)
    
    x_space = (0, 11, 1000)
    x_value = np.linspace(*x_space) 
    y = sigmoid_mod_normalized(x_value)
    y_mean = sigmoid_mod_normalized(x_value) * mean_cycle_life / results_cycle_life
    
    # Find the window indexes based on the relative size compared to results_cycle_life
    window_last_idx = (np.abs(y - relative_results[0])).argmin()
    window_first_idx = (np.abs(y - relative_results[0] + relative_window_size)).argmin()
    
    # Build a mask for the window
    window_mask = np.zeros_like(y, dtype=bool)
    window_mask[window_first_idx : window_last_idx] = True
    
    
    x_plot = np.linspace(100, 80, 1000)
    
    sigmoid_trace = go.Scatter(dict(
            x=y, 
            y=x_plot, 
            mode='lines',
            name='Capacity rendering',
            line=dict(color = 'rgba(51, 204, 204, 0.8)', width=3)))
    window_trace = go.Scatter(dict(
            x=y[window_mask], 
            y=x_plot[window_mask], 
            mode='lines',
            name='Cycle window',
            line=dict(color = 'rgba(255, 153, 0, 1.0)', width=3),
            showlegend=False))
    mean_trace = go.Scatter(dict(
            x=y_mean, 
            y=x_plot, 
            mode='lines',
            line=dict(color = 'rgba(51, 204, 204, 0.2)', width=3),
            name='Average cycle'),
            xaxis='x2')
    dot_trace = go.Scatter(dict(
            x=[relative_results[0]], 
            y=[x_plot[window_last_idx]], 
            mode='markers',
            name='You are here!',
            marker=dict(
                color = 'rgba(255, 153, 0, 1.0)',
                size = 20)))
    
    layout = go.Layout(
        plot_bgcolor='rgb(250, 250, 250)',
        xaxis=dict(
            tickmode='array',
            tickvals=[relative_results[0], 1],
            ticktext = [int(results[0] * scaling_factors[cst.REMAINING_CYCLES_NAME]),
                        str(results_cycle_life)],
            ticklen=2,
            tickcolor='rgb(250, 250, 250)',
            zeroline=False,
            range=[-0.01, max(y.max(), y_mean.max()) + 0.03],
            showgrid=False,
            title = "Number of cycles",
            titlefont=dict(family='Arial', size=24),
            tickfont=dict(family='Arial', size=22, color='rgba(255, 133, 51, 1.0)')
        ),
        xaxis2=dict(
            overlaying='x',
            tickmode='array',
            tickvals=[y_mean.max()],
            ticktext = [str(mean_cycle_life)],
            ticklen=5,
            tickcolor='rgb(250, 250, 250)',
            zeroline=False,
            range=[-0.01,  max(y.max(), y_mean.max()) + 0.03],
            showgrid=False,
            tickfont=dict(family='Arial', size=18, color='rgba(15, 15, 15, 0.5)')
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=[80, 100],
            ticktext = ["80%", "100%"],
            dtick=20,
            ticklen=5,
            tickcolor='rgb(255, 255, 255)',
            range=[80, 100],
            showgrid=False,
            title = "Relative capacity",
            titlefont=dict(family='Arial', size=24),
            tickfont=dict(family='Arial', size=18)
        ), 
        shapes = [
        # Line Vertical
        {
            'type': 'line',
            'x0': relative_results[0],
            'y0': 80,
            'x1': relative_results[0],
            'y1': x_plot[window_last_idx],
            'line': {'color': 'rgb(150, 150, 150)',
                     'width': 1,
                     'dash': 'dot'},
        }]
    )
    
    fig = go.Figure(data=[sigmoid_trace, window_trace, mean_trace, dot_trace], layout=layout)
    fig['layout'].update(height=600, width=1000)
    
    if inline:
        pyo.iplot(fig)
    else:
        pyo.plot(fig)

window_size = 20
visualize_results(results, window_size, scaling_factors, mean_cycle_life, inline=True)

relative results [0.2 0.8]
window 0.013333333333333334
